## Agent build in Langraph, integrating tools for Brave Search and other functionalities

### Langgraph

Langgraph is a framework for building agents and workflows. There are many useful pre-built functions we utilise in this example.


In [33]:
# Imports
from langchain_openai import ChatOpenAI

In [17]:
# Load env variables and instantiate LLM
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [31]:
from langchain_core.tools import tool
from typing import Optional, List, Dict, Any
from langchain.tools import tool

# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

@tool
def brave_search(query: str, result_count: int = 1) -> List[Dict[str, Any]]:
    """
    Searches BraveSearch for the recent crypto information/news
    
    Args:
        query: The search query for crypto info (The market sentiment, recent legislation etc.)
        result_count: The number of results to return
        
    Returns:
        A list of news articles with titles, desc, URL, and published date
    """
    import asyncio
    import aiohttp
    from dotenv import load_dotenv
    import os
    
    load_dotenv()
    BRAVE_SEARCH_API_KEY = os.getenv("BRAVE_SEARCH_API_KEY")

    async def _brave_search():
        async with aiohttp.ClientSession() as session:
            url = "https://api.search.brave.com/res/v1/news/search"
            headers = {
                "Accept": "application/json",
                "Accept-encoding": "gzip", 
                "X-Subscription-Token": BRAVE_SEARCH_API_KEY,
            }
            params = {"q": query,
                      "freshness": "pd"}
            
            async with session.get(url, headers=headers, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    extracted_data = []
                    for item in data.get("results", [])[:result_count]:
                        extracted_data.append({
                            "title": item.get("title", "N/A"),
                            "url": item.get("url", "N/A"),
                            "description": item.get("description", "N/A"),
                            "page_age": item.get("page_age", "N/A"),
                            "age": item.get("age", "N/A"),
                            "extra_snippets": item.get("extra_snippets", []),
                        })
                    return extracted_data
                else:
                    return [{"error": f"Request failed with status code {response.status}"}]

    return asyncio.run(_brave_search())

tools = [brave_search]

In [34]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

# Pass in:
# (1) the augmented LLM with tools
# (2) the tools list (which is used to create the tool node)
pre_built_agent = create_react_agent(llm, tools=tools)

# Invoke
messages = [HumanMessage(content="What is the crypto market sentiment for solana?")]
messages = pre_built_agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is the crypto market sentiment for solana?
================================== Ai Message ==================================
Tool Calls:
  brave_search (call_x0Hd4T9NweYlQANlVLMFcpZW)
 Call ID: call_x0Hd4T9NweYlQANlVLMFcpZW
  Args:
    query: Solana crypto market sentiment
    result_count: 5
================================= Tool Message =================================
Name: brave_search

[{"title": "Solana (SOL) Skyrockets 9% as Crypto Sentiment Turns Positive", "url": "https://u.today/solana-sol-skyrockets-9-as-crypto-sentiment-turns-positive-0", "description": "After a dip on Thursday, Solana resumed its rebound on Friday, reaching intraday highs of $125.49 in the early Saturday trading session. The rebound in SOL price coincides with an overall positive shift in sentiment across the crypto market with some crypto tokens attracting interest in the ...", "page_age": "2025-04-12T12:37:00", "age": 